In [16]:
from itertools import combinations
import numpy as np
import icecream as ic

## Simple Test Problem

In [17]:
CITIES = [
    "Rome",
    "Milan",
    "Naples",
    "Turin",
    "Palermo",
    "Genoa",
    "Bologna",
    "Florence",
    "Bari",
    "Catania",
    "Venice",
    "Verona",
    "Messina",
    "Padua",
    "Trieste",
    "Taranto",
    "Brescia",
    "Prato",
    "Parma",
    "Modena",
]
test_problem = np.load('./lab2/test_problem.npy')

## Common tests

In [18]:
problem = np.load('lab2/problem_r2_100.npy')

In [19]:
# Negative values?
np.any(problem < 0)

np.True_

In [20]:
# Diagonal is all zero?
np.allclose(np.diag(problem), 0.0)

False

In [21]:
# Symmetric matrix?
np.allclose(problem, problem.T)

False

In [22]:
# Triangular inequality
all(
    problem[x, y] <= problem[x, z] + problem[z, y]
    for x, y, z in list(combinations(range(problem.shape[0]), 3))
)

False

In [23]:
def cost(adj, sol):

    s = np.asarray(sol, dtype=int)
    to_idx = np.roll(s, -1)
    return float(np.sum(adj[s, to_idx]))

In [24]:
def starting_point(problem):
    n = problem.shape[0]
    rng = np.random.default_rng()
    return rng.permutation(np.arange(0, n))

In [25]:
def swap_mutation(solution, mutation_rate=0.05):
    """
    swap mutation with probability: mutation_rate
    """
    new_solution = solution.copy()
    rng = np.random.default_rng()
    
    if rng.random() < mutation_rate:
        n = len(new_solution)
        idx1, idx2 = rng.integers(0, n, 2)
        while idx1 == idx2:
            idx2 = rng.integers(0, n)
            
        new_solution[idx1], new_solution[idx2] = new_solution[idx2], new_solution[idx1]
        
    return new_solution

In [26]:
def order_crossover_ox1(parent1, parent2):
    """
    Crossover OX1
    mantain certain parent1 gene and fill the splution with the remanent parent2 gene in order
    """
    rng = np.random.default_rng()
    n = len(parent1)
    
    # 1. Choose two cut points
    cut1, cut2 = rng.integers(0, n, 2)
    if cut1 > cut2:
        cut1, cut2 = cut2, cut1
    if cut1 == cut2:
        cut2 += 1 # minimun 1 elemente
        
    child = -np.ones(n, dtype=int)
    
    # 2.copy of parent1 genes
    child[cut1:cut2] = parent1[cut1:cut2]
    
    # 3. vector for parent2 genes
    parent2_genes = []
    
    genes_in_child = set(child[cut1:cut2])
    for gene in parent2:
        if gene not in genes_in_child:
            parent2_genes.append(gene)
            
    # 4. fill child with parent2 gene only if parent1 gene is not present
    child_idx = cut2
    parent2_idx = 0
    
    while parent2_idx < len(parent2_genes):
        if child_idx == n:
            child_idx = 0
            
        if child_idx == cut1:
            child_idx = cut2
        
        child[child_idx] = parent2_genes[parent2_idx]
        child_idx += 1
        parent2_idx += 1
        
    return child

In [27]:
def tournament_selection(population, costs, k=3):
    """
    choose random k element in population and return the best one
    """
    rng = np.random.default_rng()
    
    indices = rng.choice(len(population), k, replace=False)
    
    best_cost = np.inf
    best_index = -1
    
    for idx in indices:
        if costs[idx] < best_cost:
            best_cost = costs[idx]
            best_index = idx
            
    return population[best_index]

In [28]:
def evolutionary_algorithm(problem, population_size, num_generations, 
                           tournament_size=3, mutation_rate=0.05, elitism=1, verbose=True):
    
    # === 1. Starting ===
    if verbose:
        print(f"Start EA: Population={population_size}, Genaration={num_generations}")
    population = [starting_point(problem) for _ in range(population_size)]
    costs = [cost(problem, ind) for ind in population]
    
    best_global_sol = population[np.argmin(costs)]
    best_global_cost = np.min(costs)

    if verbose:
        print(f"Geneartion 0: cost = {best_global_cost:.2f}")

    # === 2. EA ===
    for gen in range(1, num_generations + 1):
        new_population = []
        
        # Elitism
        sorted_indices = np.argsort(costs)
        for i in range(elitism):
            elite_index = sorted_indices[i]
            new_population.append(population[elite_index])
            
        # --- New genereration ---
        while len(new_population) < population_size:
            # a. Parents selection
            parent1 = tournament_selection(population, costs, k=tournament_size)
            parent2 = tournament_selection(population, costs, k=tournament_size)
            
            # b. Crossover
            offspring = order_crossover_ox1(parent1, parent2)
            
            # c. Mutation
            offspring = swap_mutation(offspring, mutation_rate=mutation_rate)
            
            new_population.append(offspring)
            
        population = new_population
        costs = [cost(problem, ind) for ind in population]
        
        # Update best Global value
        current_best_cost = np.min(costs)
        if current_best_cost < best_global_cost:
            best_global_cost = current_best_cost
            best_global_sol = population[np.argmin(costs)]
            
        if gen % 100 == 0 or gen == num_generations:
            if verbose:
                print(f"Generation {gen}: cost = {best_global_cost:.2f}")

    if verbose:
        print("End EA.")
    return best_global_sol, best_global_cost

In [ ]:
import os
import glob

def run_single_experiment(filepath, pop_size=100, generations=1000,
                          mutation_rate=0.1, tournament_k=20, elitism_count=5,
                          verbose=True):
    filename = os.path.basename(filepath)
    
    if verbose:
        print(f"Processing: {filename}...")

    try:
        problem = np.load(filepath)

        # Execution EA
        _, best_ea_cost = evolutionary_algorithm(
            problem,
            population_size=pop_size,
            num_generations=generations,
            tournament_size=tournament_k,
            mutation_rate=mutation_rate,
            elitism=elitism_count,
            verbose=verbose
        )

        random_sol = starting_point(problem)
        random_cost = cost(problem, random_sol)

        if verbose:
            print(f" -> Done. EA: {best_ea_cost:.2f} | Random: {random_cost:.2f}")

        return {
            'filename': filename,
            'ea_cost': best_ea_cost,
            'random_cost': random_cost,
            'error': None
        }

    except Exception as e:
        if verbose:
            print(f" -> ERROR: {e}")
        return {
            'filename': filename,
            'ea_cost': None,
            'random_cost': None,
            'error': str(e)
        }


In [ ]:
input_folder='lab2'
output_file='result.txt'
problem_files = sorted(glob.glob(os.path.join(input_folder, '*.npy')))
if not problem_files:
    print(f"Error")

with open(output_file, 'w') as f:
    f.write(f"{'File Name':<30} | {'EA Cost':<15} | {'Random Cost':<15}\n")
    f.write("-" * 65 + "\n")

    for filepath in problem_files:
        print(f"Processing: {filepath}...")
        res = run_single_experiment(filepath, verbose=False)

        if res['error'] is None:
            line = f"{res['filename']:<30} | {res['ea_cost']:<15.2f} | {res['random_cost']:<15.2f}\n"
        else:
            line = f"{res['filename']:<30} | {'ERROR':<15} | {res['error']}\n"

        f.write(line)
        f.flush()

    print(f"\nBatch completed.")

In [ ]:

risultato = run_single_experiment('lab2/problem_g_100.npy')